# ENEXIS Graduation Project

# CRISP-DM Phase 2: Data Understanding - CBS demographic Data

### Collect Initial Data

In [ ]:
import cbsodata
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nbconvert
pd.set_option('display.max_rows', 150)
pd.options.mode.chained_assignment = None  # default='warn'

Lets select a data set from CBS. One of the most comprehensive sets is "Kerncijfers wijken en buurten" which is actualized every year. It contains demographical data but also some data related to energy consumption, which can be linked to Enexis data via CBS area codes (Gemeente, Wijk and Buurt level). "Kerncijfers wijken en buurten 2019" is the most recent set containing largely complete data. In the sets of 2020 and 2021 many feature columns are yet empty.

In [ ]:
datasets = (
'85039NED', #Kerncijfers wijken en buurten 2021
'84799NED', #Kerncijfers wijken en buurten 2020
'84583NED', #Kerncijfers wijken en buurten 2019
'84286NED', #Kerncijfers wijken en buurten 2018
'83765NED', #Kerncijfers wijken en buurten 2017
'83487NED' #Kerncijfers wijken en buurten 2016
)

In [ ]:
data_years = {'85039NED' : 2021,
              '84799NED' : 2020,
              '84583NED' : 2019, 
              '84286NED' : 2018,
              '83765NED' : 2017,
              '83487NED' : 2016}

### Create a dataset with number of households by buurt by year

First, make a list of gemeenten in Enexis zone

In [ ]:
c_path = "../Data/"
v_file = "Gemeenten_Enexis" 

In [ ]:
gem_Enexis     = pd.read_csv(filepath_or_buffer = c_path + v_file + ".csv",
                      encoding           = 'UTF-8')

In [ ]:
gem_Enexis = gem_Enexis['0'].tolist()

Uploading CBS Kerncijfers datasets of 2016 - 2021, merging them, filtering out gemeenten belonging to the Enexis zone and selecting as features only number of households and the variable indicating changes in buurt delimitation ("IndelingswijzigingWijkenEnBuurten_4"). 

In [ ]:
# This function only needs to be run once, to read data from all of the CBS Kerncijfers datasets, and save them to a csv file. 
# Note it takes quite some time to run.

def households():
    df_households = pd.DataFrame()
    for i in range(len(datasets)):
        df_new = pd.DataFrame(cbsodata.get_data(datasets[i]))
        df_new = df_new.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
        df_new = df_new[df_new['SoortRegio_2'] == 'Buurt']
        df_new = df_new[df_new['Gemeentenaam_1'].isin(gem_Enexis)]
        df_new = df_new.loc[:,['Codering_3','HuishoudensTotaal_28', 'IndelingswijzigingWijkenEnBuurten_4']]
        df_new['Year'] = data_years[datasets[i]]
        df_households = pd.concat([df_households, df_new], axis = 0)
    df_households.to_csv(path_or_buf = c_path + 'CBS_Households.csv', index = False)

In [ ]:
df_households     = pd.read_csv(filepath_or_buffer = c_path + 'CBS_Households' + ".csv",
                      encoding           = 'UTF-8')

In [ ]:
df_households = df_households.sort_values(['Codering_3', 'Year'])

In [ ]:
df_households.head()

In [ ]:
df_households.info()

In [ ]:
df_households.describe(include = 'all')

In [ ]:
df_households.isna().sum()

In [ ]:
df_households['IndelingswijzigingWijkenEnBuurten_4'].value_counts()

Only buurten with no changes in demarcation will be taken into account (variable IndelingswijzigingWijkenEnBuurten_4 must be equal to 1 or be missing value).

In [ ]:
df_households = df_households[df_households['IndelingswijzigingWijkenEnBuurten_4'] == 1]

In [ ]:
df_households.describe(include = 'all')

In [ ]:
# Creating a table with number of households by year in columns, and the buurt as index
df_households_pivot = df_households.pivot(index='Codering_3', columns='Year')['HuishoudensTotaal_28']

In [ ]:
df_households_pivot.info()

In [ ]:
df_households_pivot.head()

In [ ]:
df_households_pivot.describe()

In [ ]:
df_households_pivot.isna().sum()

Very large number of buurten has missing number of households, and the number of missing households increases moving back in time. It may be explained by changes in buurten introduced over time. It may have happened that buurten have been merged, divided or borders have been changed, after which a buurt was given a new name and / or code. As a starting point, we will only consider buurten which existed in the freshest dataset, which is 2021, and all other buurten will be deleted. 

In [ ]:
df_households_pivot = df_households_pivot[df_households_pivot[2021].notna()]

Then, we shall select only these, for which the household number future had a non-zero value throughout all the years in the range 2016 - 2021. In these way we ensure that we base our model only on buurten which existed in the same borders in this time period, which means we drop all records with missing values. 

In [ ]:
df_households_pivot = df_households_pivot.dropna()

In [ ]:
df_households_pivot.info()

We see that out of original 4934 buurten only 3015 remains. There may exist smarter ways to avoid losing almost 40% of all buurten, but we leave it for a later time to refine. For now, we have more than 3000 buurten with continuity of delimitation, which looks still like a fair figure to build a model upon. As the next thing, we shall investigate the number of households in the buurten, and filter out these which have the number of households of 0 or close. We are interested in residential areas, so those with extremy low number of households are probably not truly residential, and moreover our interest is in the PV installed per 100 households, so buurten with such low population would not really matter. Let's first check for buurten with 0 households. 

In [ ]:
# Buurten with 0 households
df_households_pivot[df_households_pivot[2021] == 0].info()

In [ ]:
# Buurten with number of households of 10 and lower.
df_households_pivot[df_households_pivot[2021] <= 10].info()

In [ ]:
df_households_pivot[df_households_pivot[2021] <= 10].hist(bins = 50)
plt.show()

It seems that with some exceptions these are the same buurten over the years. We shall remove all buurten for which the number of households in 2021 was 10 or lower.

In [ ]:
df_households_pivot = df_households_pivot[df_households_pivot[2021] > 10]

In [ ]:
df_households_pivot.info()

Now, we will introduce a new column showing the relative increase (or decrease) of the number of households over time

In [ ]:
df_households_pivot['Household_increase'] = (df_households_pivot[2021] - df_households_pivot[2016]) / df_households_pivot[2021]

In [ ]:
df_households_pivot['Household_increase'].hist(bins = 50, range=(-0.5, 0.5))

Let's have a brief look at the buurten where the increase was high (e.g. higher than by 20%)

In [ ]:
df_households_increasing = df_households_pivot[df_households_pivot['Household_increase'] > 0.2]

In [ ]:
df_households_increasing.head()

In [ ]:
df_households_increasing.describe()

In [ ]:
df_households_increasing.mean()[:-1].plot()

As expected, these are buurten which show on average a steady household number increase over the years. We are interested in "static" buurten, so we shall filter out these neighborhoods where the change (increase or decrease) in household number is not bigger than 10 %.

In [ ]:
df_households_pivot = df_households_pivot[df_households_pivot['Household_increase'].between(-0.1, 0.1)]

In [ ]:
df_households_pivot.info()

In [ ]:
df_households_pivot.mean()[:-1].plot()

In [ ]:
df_households_pivot.head()

Finally, let's make a list of these "static" neighborhoods. Our further EDA and modeling will refer to these neighborhoods only.

In [ ]:
buurten_actual = df_households_pivot.index.tolist()

In [ ]:
buurten_actual

We will save the list into a new file, 'Gemeenten_Enexis_final.csv'

In [ ]:
pd.DataFrame(buurten_actual).to_csv(path_or_buf = c_path + 'Buurten_Enexis_final.csv', index = False)

### Preparation of 2019 Kerncijfers dataset

In [ ]:
selected_dataset = datasets[2]

In [ ]:
df_orig = pd.DataFrame(cbsodata.get_data(selected_dataset))

In [ ]:
df_orig.shape

In [ ]:
df_orig.head(5).transpose()

In [ ]:
#remove whitespaces from beginning and end of string column labels
df_orig = df_orig.apply(lambda x: x.str.strip() if x.dtype == "object" else x)   

### Create a subset of 2019 data for further EDA and modeling

For further steps, a subset will be created with selected features, considered to be relevant for PV installations. The analysis will be done on buurt level so only buurt level data will be selected.

In [ ]:
df = df_orig[df_orig['SoortRegio_2'] == 'Buurt']

Now, we shall take the subset of the data, with our selected, "static" buurten of the Enexis zone.

In [ ]:
df = df[df['Codering_3'].isin(buurten_actual)]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['Gemeentenaam_1'].nunique()

In [ ]:
df['Gemeentenaam_1'].value_counts().head()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

In order to use in the EDA, numeric features have to be intensive variables i.e. such which do not depend on the size of the system. For this reason, two new variables are created by dividing the existing extensive variables `AantalInkomensontvangers_70` and `BedrijfsvestigingenTotaal_91` by number of inhabitants and number of households, respectively. Another new variable is defined to describe average education level, as a kind of weighted average of `OpleidingsniveauLaag_64`, `OpleidingsniveauMiddelbaar_65` and `OpleidingsniveauHoog_66`, with respective weights of 1, 2 and 3. The new variable assumes values in the range of 1 to 3.

In [ ]:
df['Gemiddeld_opleidingsniveau'] = (df['OpleidingsniveauLaag_64'] * 1 + df['OpleidingsniveauMiddelbaar_65'] * 2 + 
                                df['OpleidingsniveauHoog_66'] * 3) / (df['OpleidingsniveauLaag_64'] + 
                                df['OpleidingsniveauMiddelbaar_65'] + df['OpleidingsniveauHoog_66'])

In [ ]:
df['Percent_inkomensontvangers'] = df['AantalInkomensontvangers_70'] / df['AantalInwoners_5']

In [ ]:
df['Bedrijfsvestigingen_per_huishuidens'] = df['BedrijfsvestigingenTotaal_91'] / df['HuishoudensTotaal_28']

We now make a slice of the dataframe df, containing only the selected features, and the newly added features. The features are stored in a list object `Selected_CBS_features`.

In [ ]:
Selected_CBS_features = ['WijkenEnBuurten',
 'Gemeentenaam_1',
 'SoortRegio_2',
 'Codering_3',
 'MeestVoorkomendePostcode_113',
 'HuishoudensTotaal_28',
 'GemiddeldeHuishoudensgrootte_32',
 'Bevolkingsdichtheid_33',
 'Woningvoorraad_34',
 'GemiddeldeWoningwaarde_35',
 'PercentageEengezinswoning_36',
 'Koopwoningen_40',
 'InBezitWoningcorporatie_42',
 'InBezitOverigeVerhuurders_43',
 'BouwjaarVanaf2000_46',
 'GemiddeldElektriciteitsverbruikTotaal_47',
 'GemiddeldAardgasverbruikTotaal_55',
 'Gemiddeld_opleidingsniveau',
 'Percent_inkomensontvangers',
 'Bedrijfsvestigingen_per_huishuidens',
 'PersonenautoSPerHuishouden_102',
 'AfstandTotSchool_108',
 'MateVanStedelijkheid_115']

In [ ]:
df = df[Selected_CBS_features]

### Description of data

We now have the final set of features extracted from CBS dataset. Let's explore its main characteristics.

In [ ]:
df.info()

There is a certain amount of missing values, which will need to be handled later. For now we will perform an initial EDA with the set as it is.

In [ ]:
df.describe(include = 'all').transpose()

In [ ]:
df.head().transpose()

Lets first have a look on the distribution of the individual features, using a set of histograms.

In [ ]:
df.drop('Codering_3', axis = 1).hist(bins=25, figsize=(16,14));

Finally, we write the dataset to a csv file, so that it can be used as input file in next stages of analysis.

In [ ]:
c_path ="../Data/"

In [ ]:
df.to_csv(path_or_buf = c_path + 'CBS_Kerncijfers2019_intermediate_dataset.csv', index = False)

In [ ]:
#nbconvert.PDFExporter('Enexis CBS data EDA v1')